In [17]:
import pickle
from pathlib import Path

import matplotlib.pyplot as plt
import torch
from neuralhydrology.evaluation import metrics
from neuralhydrology.nh_run import start_run, eval_run

In [15]:
from file_conversion import DataFrame_to_CDF
from tqdm import tqdm
import pandas as pd

# Read the list of basin IDs from the text file
with open('c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/basin_list/Canada_full_basin_list.txt', 'r') as file:
    basin_ids = file.read().splitlines()

output_path = 'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/data/time_series'

for basin_id in tqdm(basin_ids):
    # Construct the file path for the CSV file
    csv_file_path = f'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/Canadian_climate_discharge_data/{basin_id}.csv'
    
    # Read the CSV file into a DataFrame
    
    df = pd.read_csv(csv_file_path, index_col='Date')
    df.index.name = 'date'
    df.index = pd.to_datetime(df.index)
    
    if df.isnull().values.any():
        print(f"Skipping basin {basin_id} due to missing values")
        print(f"number of missing values: {df.isnull().sum().sum()}")
        print(f"rows with missing values: {df[df.isnull().any(axis=1)].index}")
        print(f"columns with missing values: {df.columns[df.isnull().any()]}")
        raise ValueError("Missing values found in the DataFrame")
    
    # Convert the DataFrame to a CDF file
    DataFrame_to_CDF(df, output_path, basin_id)
    

100%|██████████| 79/79 [00:02<00:00, 34.38it/s]


In [22]:
import os
import pandas as pd

# Define the directory containing the imputed CSV files
directory = 'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/US_climate_discharge_data'

# Initialize a flag to check if all files meet the condition
all_files_valid = True

with open('c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/basin_list/US_full_basin_list.txt', 'r') as file:
    basin_ids = file.read().splitlines()

# Iterate over each file in the directory
for basin_id in tqdm(basin_ids):
    
    file_path = f'c:/Users/ybrot/Desktop/course/UROP/GAGEii_modeling/US_climate_discharge_data/{basin_id}.csv'
    file_name = f'{basin_id}.csv'
        
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    df.rename(columns={"Date": "date"}, inplace=True)
    # Ensure the 'date' column is in datetime format and set as index
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df.set_index('date', inplace=True)
    
    # Create a full date range and check for missing dates
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')
    missing_dates = full_range.difference(df.index)
    
    # Check if the frequency of missing dates is 'D'
    if missing_dates.freq != 'D':
        print(f"File {file_name} does not meet the condition.")
        print(f"Frequency of missing dates: {missing_dates.freq}")
        all_files_valid = False
        break

if all_files_valid:
    print("All files meet the condition.")
else:
    print("Some files do not meet the condition.")

  0%|          | 0/131 [00:00<?, ?it/s]

File 04015330.csv does not meet the condition.
Frequency of missing dates: None
Some files do not meet the condition.


In [23]:
import pandas as pd

# Load the CSV file
file_path = '04015330.csv'
df = pd.read_csv(file_path)

# Ensure the 'date' column is in datetime format
df['date'] = pd.to_datetime(df['date'])

# Set the 'date' column as the index
df.set_index('date', inplace=True)

# Generate a date range from the minimum to the maximum date in the dataframe
date_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq='D')

# Reindex the dataframe to this date range
df_reindexed = df.reindex(date_range)

# Find missing dates
missing_dates = df_reindexed[df_reindexed.isnull().any(axis=1)].index

# Print missing dates
print("Missing dates:")
for date in missing_dates:
    print(date.strftime('%Y-%m-%d'))

# Check frequency
frequency = pd.infer_freq(df.index)
print(f"Detected frequency: {frequency}")

FileNotFoundError: [Errno 2] No such file or directory: '04015330.csv'

In [18]:
start_run(config_file=Path('config.yml'))

2025-02-02 06:32:22,465: Logging to c:\Users\ybrot\Desktop\course\UROP\GAGEii_modeling\runs\GAGEii-CanadianBasins-Attempt2_0202_063222\output.log initialized.
2025-02-02 06:32:22,466: ### Folder structure created at c:\Users\ybrot\Desktop\course\UROP\GAGEii_modeling\runs\GAGEii-CanadianBasins-Attempt2_0202_063222
2025-02-02 06:32:22,466: ### Run configurations for GAGEii-CanadianBasins-Attempt2
2025-02-02 06:32:22,467: experiment_name: GAGEii-CanadianBasins-Attempt2
2025-02-02 06:32:22,467: run_dir: c:\Users\ybrot\Desktop\course\UROP\GAGEii_modeling\runs\GAGEii-CanadianBasins-Attempt2_0202_063222
2025-02-02 06:32:22,467: train_basin_file: basin_list\train.txt
2025-02-02 06:32:22,468: validation_basin_file: basin_list\validation.txt
2025-02-02 06:32:22,469: test_basin_file: Canadian_basin_list_test.txt
2025-02-02 06:32:22,469: train_start_date: 1981-01-01 00:00:00
2025-02-02 06:32:22,469: train_end_date: 2022-12-31 00:00:00
2025-02-02 06:32:22,470: validation_start_date: 1981-01-01 00:0

C:\Users\ybrot\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\neuralhydrology\datautils\utils.py:242: SyntaxWarning: invalid escape sequence '\d'
  weekly_freq = re.match('(\d+)W(-(MON|TUE|WED|THU|FRI|SAT|SUN))?$', native_frequency)


ValueError: Cannot infer a legal frequency from dataset: None.